In [ ]:
!pip install pandas scipy numpy matplotlib

In [ ]:
import glob
import pandas as pd
import os
import scipy.stats as stats
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
'''EA TASK VIDEO SEPARATION - THIS WILL PRODUCE NINE CSV'S WHICH CONSIDERS THE VIDEO SPLIT'''

#circle_starts = [91, 190, 351, 445, 612, 725]

# Define the specific timepoint intervals for the nine segments
segments = [
    (0, 91),    # first
    (112, 190), #second
    (211, 273), #third
    (274, 351), #fourth
    (372, 445), #fifth
    (466, 546), #sixth
    (547, 612), #seventh
    (633, 725), #eigth
    (746, 819)  #ninth
]

# Reading the files
files_EA_resid_ts = sorted(glob.glob("/projects/tsecara/EA_GLM_SPINS_ASD/data/SPINS_EA_RS_cleaned/SPASD/**/sub-*_RS_2mm_GSR_glasser_tian_meants_final.csv", recursive = True))

# Getting the region names
data = pd.read_csv("/projects/tsecara/SPINS_ASD_Project2/G_atlas_info_reorder_correct.csv")
list_of_region_names = data['atlas_roi'].tolist()

# Filtering out empty files
filtered_files = [file for file in files_EA_resid_ts if os.path.getsize(file) != 0]

# Creating the participant ID list
ptlist = ["SPASD_" + file[82:85] + "_" + file[85:89] for file in files_EA_resid_ts]

# Reading the time series files
resid_ts = [pd.read_csv(file, header=None) for file in filtered_files]

# Naming the dictionary with the participant IDs
resid_ts_named = {name: df for name, df in zip(ptlist, resid_ts)}

# Define the total number of time points
total_timepoints = 819  # Adjust this based on your data

# Create the output directory if it doesn't exist
output_dir = "/projects/tsecara/SPINS_ASD_Project2/data/SPASD/Circles_SPASD_Participant_9_CSVs/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Function to extract segments from a time series dataframe
def extract_segments(df, segments):
    segment_dfs = []
    for i, (start, end) in enumerate(segments):
        segment_df = df.iloc[:, start:end]
        segment_dfs.append(segment_df)
    return segment_dfs

# Extracting segments and saving them as separate CSV files
for pt_id, df in resid_ts_named.items():
    segment_dfs = extract_segments(df, segments)
    for i, segment_df in enumerate(segment_dfs):
        output_path = os.path.join(output_dir, f"{pt_id}_segment_{i+1}.csv")
        segment_df.to_csv(output_path, index=False, header=False)

print("done")


In [ ]:
# Read the files and filter out empty files
circle_files = sorted(glob.glob("/projects/tsecara/SPINS_ASD_Project2/data/SPASD/Circles_SPASD_Participant_9_CSVs/*.csv", recursive=True))
non_empty_files = [file for file in circle_files if os.path.getsize(file) != 0]

# Create the participant ID list based on file names
participant_ids = sorted(list(set(file.split('_segment_')[0].split('/')[-1] for file in non_empty_files)))

# Read the time series files for each participant and segment
time_series_data = {pt_id: [] for pt_id in participant_ids}
for file in non_empty_files:
    participant_id = file.split('_segment_')[0].split('/')[-1]
    df = pd.read_csv(file, header=None)
    time_series_data[participant_id].append(df)

In [ ]:
''Reads non-empty CSVs, extracts participant IDs, and stores each participant’s time series data in a dictionary. '''

# Read the files and filter out empty files
circle_files = sorted(glob.glob("/projects/tsecara/SPINS_ASD_Project2/data/SPASD/Circles_SPASD_Participant_9_CSVs/*.csv", recursive=True))
non_empty_files = [file for file in circle_files if os.path.getsize(file) != 0]

# Create the participant ID list based on file names
participant_ids = sorted(list(set(file.split('_segment_')[0].split('/')[-1] for file in non_empty_files)))

# Read the time series files for each participant and segment
#This will identify participants that contain issues in their code, so they can be excluded later on 
time_series_data = {pt_id: [] for pt_id in participant_ids}
for file in non_empty_files:
    participant_id = file.split('_segment_')[0].split('/')[-1]
    try:
        df = pd.read_csv(file, header=None)
        if df.empty:
            print(f"Warning: File {file} is empty.")
        else:
            time_series_data[participant_id].append(df)
    except pd.errors.EmptyDataError:
        print(f"Error: File {file} is empty or cannot be read.")

# Print the result to verify
print("Time series data collected for participants:")
for pt_id, data in time_series_data.items():
    print(f"Participant {pt_id}: {len(data)} files")


In [ ]:
'''GETS THE MSSD VALUES HERE FOR THE WEIGHTED AVERAGE'''
# Function to mean center data
def mean_centering(segment):
    center_function = lambda x: x - x.mean()
    centered_data = center_function(segment)
    return centered_data

# Function to z-score data
def z_score(segment):
    data = np.array(segment)
    z_scores = stats.zscore(data, axis=1)  # axis = 1 is each row
    return z_scores

# Function to calculate mssd values
def mssd_value(segment_scores):
    f_diff = np.diff(segment_scores)
    f_sc_diff = (f_diff) ** 2
    mean = np.nanmean(f_sc_diff, axis=1)  # axis=1 gets it from the rows
    sq_mean = np.sqrt(mean)
    mssd_values = sq_mean
    return mssd_values

# Load region names
atlas_data = pd.read_csv("/projects/tsecara/SPINS_ASD_Project2/G_atlas_info_reorder_correct.csv")
region_names = atlas_data['atlas_roi'].tolist()

# Main calculation loop
z_scores_results = []
centered_data_results = []
mssd_z_scores = []  # for z-scoring
mssd_centered = []  # for mean-centering

# Prepare matrices to store mSSD results
mssd_matrix_z = np.zeros((len(time_series_data), len(region_names) + 1), dtype=object)
mssd_matrix_centered = np.zeros((len(time_series_data), len(region_names) + 1), dtype=object)


while True:
    try:
        # Get user input to decide mean-centered or z-scored data
        user = int(input("Would you like the data to be z_scored or mean-centered? Enter 1 for z_score and 2 for mean-centered: "))
        if user in [1, 2]:
            # Calculate mSSD values for each participant and their segments
            for i, (pt_id, segments) in enumerate(time_series_data.items()):
                weights = []
                mssd_segments = []
                
                for segment in segments:
                    if user == 1:  # Z-score
                        z_scores = z_score(segment)
                        mssd = mssd_value(z_scores)
                    else:  # Mean-center
                        centered_data = mean_centering(segment)
                        mssd = mssd_value(centered_data)
                    
                    mssd_segments.append(mssd)
                    weights.append(segment.shape[1])  # Number of columns (time points)

                # Compute weighted average mSSD
                total_time_points = sum(weights)
                weighted_mssd = sum(m * w for m, w in zip(mssd_segments, weights)) / total_time_points

                # Store the results in the matrix
                if user == 1:
                    mssd_matrix_z[i, 0] = pt_id
                    mssd_matrix_z[i, 1:] = np.round(weighted_mssd, decimals=10)
                else:
                    mssd_matrix_centered[i, 0] = pt_id
                    mssd_matrix_centered[i, 1:] = np.round(weighted_mssd, decimals=10)

            # Add region names to the matrix header
            if user == 1:
                mssd_matrix_z = np.vstack(([''] + region_names, mssd_matrix_z))
            else:
                mssd_matrix_centered = np.vstack(([''] + region_names, mssd_matrix_centered))

            # Convert to DataFrame for better display
            df_mssd = pd.DataFrame(mssd_matrix_z if user == 1 else mssd_matrix_centered)
            break
        else:
            print("Invalid input. Please enter either 1 for z_score or 2 for mean-centered (integer only): ")
    except ValueError:
        print("Invalid input. Please enter either 1 for z_score or 2 for mean-centered (integer only): ")

print(df_mssd)


In [ ]:
'''GETTING CSVs - make sure to check which matrix variable is produced'''
pd.DataFrame(df_mssd).to_csv("/projects/tsecara/SPINS_ASD_Project2/data/SPASD/mssd_meancentered_SPASD_EA_RSclean.csv") #gets a csv file here